In [6]:
# Import and use
from database_explorer import create_database_connection, get_dataframe_for_notebook

# Create connection
engine = create_database_connection()

# Load DataFrame
df = get_dataframe_for_notebook(engine, 'public', 'urban_institute_data')

# Work with DataFrame
df.head()
df.info()
df.describe()

# Your analysis code here
df.groupby('year').agg({
    'enrollment': ['mean', 'std', 'count']
})

✅ Connected to PostgreSQL successfully!
Database: milestone2 on localhost:5432
PostgreSQL Version: PostgreSQL 16.1
🔄 Loading table public.urban_institute_data as DataFrame...
✅ Successfully loaded public.urban_institute_data
�� DataFrame shape: 1594865 rows × 65 columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1594865 entries, 0 to 1594864
Data columns (total 65 columns):
 #   Column                       Non-Null Count    Dtype         
---  ------                       --------------    -----         
 0   id                           1594865 non-null  int64         
 1   data_source                  1594865 non-null  object        
 2   endpoint                     1594865 non-null  object        
 3   year                         1594865 non-null  int64         
 4   fetched_at                   1594865 non-null  datetime64[ns]
 5   created_at                   1594865 non-null  datetime64[ns]
 6   updated_at                   1594865 non-null  datetime64[ns]
 7   street_m

enrollment                    
            mean         std   count
year                                
2020         NaN         NaN       0
2021  416.720663  448.974851  127477
2022  452.818111  459.507330  227254
2023  452.718179  462.247722  226903

In [ ]:
# save as export_postgres.py
import psycopg2
import pandas as pd
import os

# Database connection
conn = psycopg2.connect(
    host="localhost",
    database="milestone2",
    user="postgres",
    password="123"
)

# Create export directory
os.makedirs("postgres_export", exist_ok=True)

# Get all table names
cursor = conn.cursor()
cursor.execute("""
    SELECT tablename FROM pg_tables 
    WHERE schemaname = 'public'
""")
tables = [row[0] for row in cursor.fetchall()]

print(f"Found {len(tables)} tables: {tables}")

# Export each table
for table in tables:
    print(f"Exporting {table}...")
    df = pd.read_sql(f"SELECT * FROM {table}", conn)
    df.to_csv(f"postgres_export/{table}.csv", index=False)
    print(f"  Exported {len(df)} rows")

# Also export schema information
cursor.execute("""
    SELECT 
        table_name,
        column_name,
        data_type,
        is_nullable,
        column_default
    FROM information_schema.columns 
    WHERE table_schema = 'public'
    ORDER BY table_name, ordinal_position
""")

schema_info = cursor.fetchall()
schema_df = pd.DataFrame(schema_info, columns=['table', 'column', 'type', 'nullable', 'default'])
schema_df.to_csv("postgres_export/schema_info.csv", index=False)

conn.close()
print("Export complete! Check the 'postgres_export' folder.")